In [1]:
import pandas as pd
import re
import requests
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
%cd /content/gdrive/MyDrive/6242_PROJECT/data

/content/gdrive/MyDrive/6242_PROJECT/data


In [4]:
%ls

amenities.csv           general-information-of-schools.csv
bus_stops.csv           listing-of-supermarkets.csv
carparks.csv            processed/
data-download.png       secondary_schools.csv
data_hawker.csv         secondary_schools.gsheet
data_hawker.gsheet      supermarkets.csv
data_malls.csv          taxi_stands.csv
data_MRT.csv            ura.csv
data_prischools.csv     ura_join_bus_stops.csv
data_prischools.gsheet


In [5]:
schools= pd.read_csv("./general-information-of-schools.csv")

In [6]:
print(schools['mainlevel_code'].value_counts())

PRIMARY                  183
SECONDARY                138
MIXED LEVELS              14
JUNIOR COLLEGE            10
CENTRALISED INSTITUTE      1
Name: mainlevel_code, dtype: int64


In [7]:
primary = schools[schools['mainlevel_code']=='PRIMARY']
secondary = schools[schools['mainlevel_code']=='SECONDARY']

In [8]:
def getLatLong(x):
  postal_code = x['postal_code']
  matches = requests.get("https://developers.onemap.sg/commonapi/search?searchVal={}&returnGeom=Y&getAddrDetails=Y&pageNum=1".format(postal_code)).json()['results']
  if len(matches)>=1:
    first_match = matches[0]
    lat,long = first_match['LATITUDE'],first_match['LONGITUDE']
    isLatLongAvailable='True'
    return (lat,long,isLatLongAvailable)
  else:
    return ("NA","NA","False")

In [9]:
getLatLong(secondary.iloc[0,:])

('1.44589068910993', '103.802398196596', 'True')

In [10]:
secondary=pd.concat([secondary,secondary.apply(getLatLong,axis=1,result_type='expand').rename(mapper={0:'lat',1:'long',2:'isLatLongAvailable'},axis=1)],axis=1)

In [11]:
secondary

,school_name,url_address,address,postal_code,telephone_no,telephone_no_2,fax_no,fax_no_2,email_address,mrt_desc,bus_desc,principal_name,first_vp_name,second_vp_name,third_vp_name,fourth_vp_name,fifth_vp_name,sixth_vp_name,dgp_code,zone_code,type_code,nature_code,session_code,mainlevel_code,sap_ind,autonomous_ind,gifted_ind,ip_ind,mothertongue1_code,mothertongue2_code,mothertongue3_code,lat,long,isLatLongAvailable
1,ADMIRALTY SECONDARY SCHOOL,http://www.admiraltysec.moe.edu.sg,31 WOODLANDS CRESCENT,737916,63651733,63654596,63652774,na,Admiralty_SS@moe.edu.sg,ADMIRALTY MRT,904,MR LAM YUI- P'NG,MR NG SONG LIM STEVEN,MR SHEIK ALAUDIN B MOHD ISMAIL,NaN,NaN,NaN,NaN,WOODLANDS,NORTH,GOVERNMENT SCHOOL,CO-ED SCHOOL,SINGLE SESSION,SECONDARY,No,No,No,No,Chinese,Malay,Tamil,1.44589068910993,103.802398196596,True
3,AHMAD IBRAHIM SECONDARY SCHOOL,http://www.ahmadibrahimsec.moe.edu.sg,751 YISHUN AVENUE 7,768928,67585384,na,67557778,na,aiss@moe.edu.sg,"CANBERRA MRT, YISHUN MRT","117, 167, 169, 800, 811, 812, 883, 856, 858, 8...",MISS TAN KE XIN,MR YAO ZHIXUAN TERENCE,MISS LEA SHU HUI,NaN,NaN,NaN,NaN,YISHUN,NORTH,GOVERNMENT SCHOOL,CO-ED SCHOOL,SINGLE SESSION,SECONDARY,No,No,No,No,Chinese,Malay,Tamil,1.43605975368804,103.829718690077,True
8,ANDERSON SECONDARY SCHOOL,http://www.andersonsec.moe.edu.sg,10 ANG MO KIO STREET 53,569206,64598303,na,64586104,na,anderson_ss@moe.edu.sg,"ANG MO KIO MRT, YIO CHU KANG MRT","265, 548, 45, 50, 72",MDM THONG MAY TENG,MDM LOW CHIA YEE,MR JONAH LIM TUCK HWAY,NaN,NaN,NaN,NaN,ANG MO KIO,NORTH,GOVERNMENT SCHOOL,CO-ED SCHOOL,SINGLE SESSION,SECONDARY,No,Yes,No,No,Chinese,Malay,Tamil,1.37434001701177,103.851554107068,True
11,ANG MO KIO SECONDARY SCHOOL,http://www.angmokiosec.moe.edu.sg,6 ANG MO KIO STREET 22,569362,64548605,na,64582425,na,AMKSS@MOE.EDU.SG,ANG MO KIO MRT,"13, 74, 76, 88, 130, 132, 133, 136, 165, 169, ...",MR SHAW SWEE TAT,MDM TONG WAI HAN,MDM LEE KEE CHIN,NaN,NaN,NaN,NaN,ANG MO KIO,SOUTH,GOVERNMENT SCHOOL,CO-ED SCHOOL,SINGLE SESSION,SECONDARY,No,No,No,No,Chinese,Malay,Tamil,1.36733710171069,103.842154973672,True
12,ANGLICAN HIGH SCHOOL,http://www.anglicanhigh.moe.edu.sg,600 UPPER CHANGI ROAD,487012,62414866,na,64414690,na,AHS@MOE.EDU.SG,TANAH MERAH MRT,"2, 9, 12, 14, 24, 31, 35, 38, 45, 47, 48",MR KHOO TSE HORNG,MR GOH CHIN WEI,MR CHIN LIH KEONG,NaN,NaN,NaN,NaN,BEDOK,EAST,GOVERNMENT-AIDED SCH,CO-ED SCHOOL,SINGLE SESSION,SECONDARY,Yes,Yes,No,No,Chinese,na,na,1.33118109596833,103.941853443058,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,YUHUA SECONDARY SCHOOL,http://www.yuhuasec.moe.edu.sg,35 JURONG WEST STREET 41,649406,65661985,na,65696689,na,YUHUA_SS@MOE.EDU.SG,LAKESIDE MRT,"49, 98, 99, 157, 180, 185, 187, 198, 334, 335,...","MR PANG CHIN KEAT, KEVIN",MR TAN WEIMING BRIAN,MR CHNG PUAY KEONG,NaN,NaN,NaN,NaN,JURONG WEST,WEST,GOVERNMENT SCHOOL,CO-ED SCHOOL,SINGLE SESSION,SECONDARY,No,No,No,No,Chinese,Malay,Tamil,1.34718626876968,103.722687235839,True
339,YUSOF ISHAK SECONDARY SCHOOL,http://www.yusofishaksec.moe.edu.sg,11 BUKIT BATOK STREET 25,658712,65009800,na,65675502,na,yiss@moe.edu.sg,BUKIT BATOK MRT,"61, 66, 77, 106, 157, 174, 178, 506, 852, 963",MDM LEE WEE LIN MAUREEN,MR WILSON TAY SWEE KIAT,NaN,NaN,NaN,NaN,NaN,BUKIT BATOK,WEST,GOVERNMENT SCHOOL,CO-ED SCHOOL,SINGLE SESSION,SECONDARY,No,No,No,No,Chinese,Malay,Tamil,1.34233921440623,103.760032628302,True
340,YUYING SECONDARY SCHOOL,http://www.yuyingsec.moe.edu.sg,47 HOUGANG AVENUE 1,538884,62827968,na,62871607,na,yuying_ss@moe.edu.sg,"KOVAN MRT, SERANGOON MRT, HOUGANG MRT","25, 51, 53, 53M, 55, 62, 87, 113, 119, 151, 854",MR LO CHEE LIN,MDM NIAM HWEE PENG,MR FOO JUNG HIN SIMON,NaN,NaN,NaN,NaN,HOUGANG,SOUTH,GOVERNMENT-AIDED SCH,CO-ED SCHOOL,SINGLE SESSION,SECONDARY,No,No,No,No,Chinese,Malay,Tamil,1.35713595302467,103.890194755287,True
343,ZHENGHUA SECONDARY SCHOOL,http://www.zhenghuasec.moe.edu.sg,91 SENJA ROAD,677741,67639455,na,67633577,na,ZHENGHU

In [12]:
# secondary.loc[:,['lat','long','isLatLongAvailable']] = secondary.apply(getLatLong,axis=1,result_type='expand')

In [13]:
secondary[secondary['isLatLongAvailable']=='True'].to_csv("./secondary_schools.csv")

In [14]:
# secondary.iloc[:10,:].apply(getLatLong,axis=1,result_type='expand').rename(mapper={0:'lat',1:'long',2:'isLatLongAvailable'},axis=1)